In [ ]:
import customtkinter as ctk
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from textblob import TextBlob
from translate import Translator
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import speech_recognition as sr
import pyttsx3

class TextPredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Text Predictor")
        self.root.geometry("800x600")
        self.root.configure(bg="#ffffff")

        # Configure customtkinter
        ctk.set_appearance_mode("light")  
        ctk.set_default_color_theme("blue")  

        self.create_widgets()

        self.model = load_model('next_words.h5')
        with open('token.pkl', 'rb') as token_file:
            self.tokenizer = pickle.load(token_file)
        self.translator = Translator(to_lang="te")

        self.engine = pyttsx3.init()

    def create_widgets(self):
        self.header_frame = ctk.CTkFrame(self.root, corner_radius=0)
        self.header_frame.pack(pady=20, fill='x')

        self.google_logo = ctk.CTkLabel(self.header_frame, text="Word Predictor with Web Scrapping & Sentiment Analysis", font=("Arial", 24, "bold"), text_color="#FF69B4")  # Pink color
        self.google_logo.pack(pady=10)

        self.entry_label = ctk.CTkLabel(self.root, text="Enter your text or use voice input:", font=("Arial", 14))
        self.entry_label.pack(pady=(10, 5))

        self.text_entry = ctk.CTkEntry(self.root, width=600, placeholder_text="Enter text here...")
        self.text_entry.pack(pady=(0, 15))

        self.voice_btn = ctk.CTkButton(self.root, text="Voice Input", command=self.voice_input, font=("Arial", 14), fg_color="#FBBC05")  # Yellow
        self.voice_btn.pack(pady=5)

        self.predict_btn = ctk.CTkButton(self.root, text="Predict Next Word", command=self.predict_next_word, font=("Arial", 14), fg_color="#4285F4")  # Blue
        self.predict_btn.pack(pady=5)

        self.translate_btn = ctk.CTkButton(self.root, text="Translate to Telugu", command=self.translate_word, state=ctk.DISABLED, font=("Arial", 14), fg_color="#34A853")  # Green
        self.translate_btn.pack(pady=5)

        self.read_output_btn = ctk.CTkButton(self.root, text="Read Output", command=self.read_output, state=ctk.DISABLED, font=("Arial", 14), fg_color="#EA4335")  # Red
        self.read_output_btn.pack(pady=5)

        self.result_label = ctk.CTkLabel(self.root, text="", font=("Arial", 14))
        self.result_label.pack(pady=(20, 0))

        self.related_words_label = ctk.CTkLabel(self.root, text="", font=("Arial", 14))
        self.related_words_label.pack(pady=(20, 0))

    def voice_input(self):
        print("Voice Input button clicked")  # Debug statement
        recognizer = sr.Recognizer()
        with sr.Microphone() as source:
            self.result_label.configure(text="Listening...")
            try:
                audio = recognizer.listen(source)
                text = recognizer.recognize_google(audio)
                self.text_entry.delete(0, ctk.END)
                self.text_entry.insert(0, text)
                self.result_label.configure(text="Voice input received. You can now predict the next word.")
            except sr.UnknownValueError:
                self.result_label.configure(text="Sorry, could not understand the audio.")
            except sr.RequestError:
                self.result_label.configure(text="Could not request results; check your network connection.")

    def predict_next_word(self):
        print("Predict Next Word button clicked")  
        text = self.text_entry.get().strip()
        if text:
            words = text.split()[-3:]
            sequence = self.tokenizer.texts_to_sequences([words])[0]
            sequence = np.array([sequence])

            preds = np.argmax(self.model.predict(sequence))
            predicted_word = ""

            for key, value in self.tokenizer.word_index.items():
                if value == preds:
                    predicted_word = key
                    break

            sentiment = TextBlob(predicted_word).sentiment

            sentiment_label = f"Sentiment - Polarity: {sentiment.polarity:.2f}, Subjectivity: {sentiment.subjectivity:.2f}"
            self.result_label.configure(text=f"Predicted next word: {predicted_word}\n{sentiment_label}")

            self.translate_btn.configure(state=ctk.NORMAL)
            self.read_output_btn.configure(state=ctk.NORMAL)
            self.predicted_word = predicted_word

            related_words = self.scrape_related_words(predicted_word)
            related_words_tfidf = self.get_related_words_tfidf(predicted_word, related_words)
            related_words_sentiments = self.get_related_words_sentiments(related_words_tfidf)
            self.display_related_words(related_words_sentiments)

    def scrape_related_words(self, query):
        search_url = f"https://www.google.com/search?q={query}"
        response = requests.get(search_url)
        soup = BeautifulSoup(response.text, "html.parser")
        search_results = soup.find_all("h3")
        related_words = [result.text for result in search_results]
        return related_words

    def get_related_words_tfidf(self, query, related_words):
        documents = [query] + related_words
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
        cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

        related_words_with_scores = [(word, score) for word, score in zip(related_words, cosine_similarities)]
        related_words_with_scores.sort(key=lambda x: x[1], reverse=True)
        return [word for word, _ in related_words_with_scores]

    def get_related_words_sentiments(self, related_words):
        sentiments = []
        for word in related_words:
            sentiment = TextBlob(word).sentiment
            sentiments.append((word, sentiment.polarity, sentiment.subjectivity))
        return sentiments

    def display_related_words(self, related_words_sentiments):
        text = "Related words and their sentiments:\n"
        for word, polarity, subjectivity in related_words_sentiments:
            text += f"{word} - Polarity: {polarity:.2f}, Subjectivity: {subjectivity:.2f}\n"
        self.related_words_label.configure(text=text)

    def translate_word(self):
        if hasattr(self, 'predicted_word'):
            translated_word = self.translator.translate(self.predicted_word)
            self.result_label.configure(text=f"Predicted next word: {self.predicted_word}\nTranslation in Telugu: {translated_word}")

    def read_output(self):
        output_text = self.result_label.cget("text") + "\n" + self.related_words_label.cget("text")
        self.engine.say(output_text)
        self.engine.runAndWait()

if __name__ == "__main__":
    root = ctk.CTk()
    app = TextPredictorApp(root)
    root.mainloop()


Voice Input button clicked
Predict Next Word button clicked
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 551ms/step
Voice Input button clicked
Predict Next Word button clicked
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
